In [1]:
from sqlalchemy import *
from sqlalchemy.orm import *
import pandas as pd
import random
import numpy as np
import os
from sqlalchemy.event import *
from gradedb import GradeDB

In [2]:
%run schema.py
db=GradeDB("create.db")
db.Create()

In [3]:
#generate students' names, university id and emails
first_names=('John','Andy','Joe','Felix','Alice','April','Reiter',
             'Emory','Miller','David','Ballin','Trotter', 'Virginia',
             'Rios','Thomas','Wheeler','James','Harrell', 'Nicolas',
             'White','Mary','Flanagan','Velda','Kitty','Iris','Violet',
            'Kara','Betty','Lilly','Troy','Crystal','Cory','Leila',
            'Kristina','Lena','Hanna','Josie','Jean','Ellie','Aliya',
            'Max','Cora','Meghan','Lucie','Ann','Orla','Lana','Tilly')
last_names=('Johnson','Smith','Williams','Hill','Morton','Hutchinson',
           'Brooks','Jensen','Rogers','Montgomery','Booth','Pham','Austin',
           'Day','Ayala','Page','Wang','Rice','Vazquez','Wolf','Abbott',
           'Mcguire','Aguilar','Henry','Ross','Cole','Pratt','Frank','Cummings',
           'Clark','Wilson','Hammond','Porter','Robles','Beck','Harvey','Benson',
           'Franklin','Bass','Lloyd','Allen','Flores','Harris','Young',
           'Bell','Miller','Stevens','Williamson','Lynch','Gough','Waters')

student_names = []
np.random.seed(123)
for i in range(100):
    full_name=np.random.choice(first_names)+' '+random.choice(last_names)
    student_names.append(full_name)
student_names=tuple(student_names)

np.random.seed(123)
s_num=np.random.randint(3000000,3099999,size=100)
s_num=s_num.astype(str)
uid=[]
email=[]
for item in s_num:
    uid.append('s'+item)
    email.append('s'+item+'@leidenuniv.nl')
uid=tuple(uid)
email=tuple(email)

#add student's information into DB
for i in range(100):
    db.addStudent(uid[i],student_names[i],email[i])

In [4]:
#view students in DB
engine0 = create_engine("sqlite:///create.db",echo=False)
session = Session(engine0)
stmt = select(Students)

for user in session.scalars(stmt):
    print(user)

Students(Universityid='s3015725', Name='Orla Lloyd', Email='s3015725@leidenuniv.nl')
Students(Universityid='s3028030', Name='Joe Flores', Email='s3028030@leidenuniv.nl')
Students(Universityid='s3017730', Name='Lilly Pham', Email='s3017730@leidenuniv.nl')
Students(Universityid='s3061404', Name='Lena Smith', Email='s3061404@leidenuniv.nl')
Students(Universityid='s3063206', Name='Ellie Lynch', Email='s3063206@leidenuniv.nl')
Students(Universityid='s3015377', Name='Harrell Pratt', Email='s3015377@leidenuniv.nl')
Students(Universityid='s3073299', Name='White Page', Email='s3073299@leidenuniv.nl')
Students(Universityid='s3046203', Name='Meghan Brooks', Email='s3046203@leidenuniv.nl')
Students(Universityid='s3089302', Name='Velda Page', Email='s3089302@leidenuniv.nl')
Students(Universityid='s3022241', Name='Kristina Austin', Email='s3022241@leidenuniv.nl')
Students(Universityid='s3065632', Name='Leila Aguilar', Email='s3065632@leidenuniv.nl')
Students(Universityid='s3055409', Name='Tilly Page

In [5]:
#generate questions
def Q_generator(size=100,s=123):
    '''default: call this function return a question df with column called Title, Text'''
    '''default: 100 rows, seed 123'''
    np.random.seed(s)
    r=np.arange(1,size+1,1).astype(str)
    r=np.random.choice(r,size,replace=False)
    Question=pd.DataFrame(columns=['Index','Title','Text'])
    title=[]
    text=[]
    index=[]
    for i in range(len(r)):
        index.append(r[i])
        title.append('Question Title '+r[i])
        text.append('Example question text '+r[i]+': A teacher adds ......')
    Question['Index']=index
    Question['Title']=title
    Question['Text']=text
    return(Question)

In [6]:
#add questions into DB
Q=Q_generator()
for i in range(len(Q)):
    db.addQuestion(Q.loc[i,'Index'],Q.loc[i,'Title'],Q.loc[i,'Text'])

In [7]:
#view questions in DB
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Question)

for user in session.scalars(stmt):
    print(user)

Question(Questionid='1', Title='Question Title 1', Text='Example question text 1: A teacher adds ......')
Question(Questionid='2', Title='Question Title 2', Text='Example question text 2: A teacher adds ......')
Question(Questionid='3', Title='Question Title 3', Text='Example question text 3: A teacher adds ......')
Question(Questionid='4', Title='Question Title 4', Text='Example question text 4: A teacher adds ......')
Question(Questionid='5', Title='Question Title 5', Text='Example question text 5: A teacher adds ......')
Question(Questionid='6', Title='Question Title 6', Text='Example question text 6: A teacher adds ......')
Question(Questionid='7', Title='Question Title 7', Text='Example question text 7: A teacher adds ......')
Question(Questionid='8', Title='Question Title 8', Text='Example question text 8: A teacher adds ......')
Question(Questionid='9', Title='Question Title 9', Text='Example question text 9: A teacher adds ......')
Question(Questionid='10', Title='Question Titl

In [8]:
#random tasks generator
def T_generator(qid,size=100,question_num=7,s=123):
    '''qid represents for all the question id in the database'''
    '''default: call this function return a task df with column called Title, Text'''
    '''default: 100 rows, seed 123'''
    '''default: every task has 7 questions'''
    np.random.seed(s)
    r=np.random.randint(1,100,size).astype(str)
    Task=pd.DataFrame(columns=['Title','Text','qid_list'])
    title=[]
    text=[]
    qid_list=[]
    for i in range(len(r)):
        title.append('Task Title '+r[i])
        text.append('Example Task text '+r[i]+': This task should ......')
        qid_list.append(np.random.choice(qid,size=question_num,replace=False))
    Task['Title']=title
    Task['Text']=text
    Task['qid_list']=qid_list
    return(Task)

In [9]:
# add tasks into DB
Task_Gen=T_generator(qid=Q["Index"])

for i in range(len(Task_Gen)):
    db.addTask(Task_Gen.loc[i,"Title"],Task_Gen.loc[i,"Text"],i,Task_Gen.loc[i,'qid_list'])

In [10]:
#view task
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Task)

for user in session.scalars(stmt):
    print(user)

Students(Taskid='0', Title='Task Title 67', Text='Example Task text 67: This task should ......')
Students(Taskid='1', Title='Task Title 93', Text='Example Task text 93: This task should ......')
Students(Taskid='2', Title='Task Title 99', Text='Example Task text 99: This task should ......')
Students(Taskid='3', Title='Task Title 18', Text='Example Task text 18: This task should ......')
Students(Taskid='4', Title='Task Title 84', Text='Example Task text 84: This task should ......')
Students(Taskid='5', Title='Task Title 58', Text='Example Task text 58: This task should ......')
Students(Taskid='6', Title='Task Title 87', Text='Example Task text 87: This task should ......')
Students(Taskid='7', Title='Task Title 98', Text='Example Task text 98: This task should ......')
Students(Taskid='8', Title='Task Title 97', Text='Example Task text 97: This task should ......')
Students(Taskid='9', Title='Task Title 48', Text='Example Task text 48: This task should ......')
Students(Taskid='10'

In [11]:
# add assignments into DB
for i in range(100):
    db.addAssignment(taskid=i,universityid=uid[i])

In [12]:
#view assignment
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Assignment)

for user in session.scalars(stmt):
    print(user)
    #db.New_Ass_Remind()

Assignment(Universityid='s3015725', Taskid='0',Assignmentid='1')
Assignment(Universityid='s3028030', Taskid='1',Assignmentid='2')
Assignment(Universityid='s3017730', Taskid='2',Assignmentid='3')
Assignment(Universityid='s3061404', Taskid='3',Assignmentid='4')
Assignment(Universityid='s3063206', Taskid='4',Assignmentid='5')
Assignment(Universityid='s3015377', Taskid='5',Assignmentid='6')
Assignment(Universityid='s3073299', Taskid='6',Assignmentid='7')
Assignment(Universityid='s3046203', Taskid='7',Assignmentid='8')
Assignment(Universityid='s3089302', Taskid='8',Assignmentid='9')
Assignment(Universityid='s3022241', Taskid='9',Assignmentid='10')
Assignment(Universityid='s3065632', Taskid='10',Assignmentid='11')
Assignment(Universityid='s3055409', Taskid='11',Assignmentid='12')
Assignment(Universityid='s3023166', Taskid='12',Assignmentid='13')
Assignment(Universityid='s3071200', Taskid='13',Assignmentid='14')
Assignment(Universityid='s3033710', Taskid='14',Assignmentid='15')
Assignment(Uni

In [13]:
# add submission into DB
for i in range(100):
    db.newSubmission(universityid = uid[i])

C:\Users\Administrator\Desktop\pywork\DB\gradedb.py:73: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  a = Submission(Evaluation_request=False,Time=pd.datetime.now())


In [14]:
#view newsub
engine0 = create_engine("sqlite:///create.db",echo=False)
session = Session(engine0)
stmt = select(Submission)

for user in session.scalars(stmt):
    print(user)

Submission(Submissionid='1', Assignmentid='1',Time='2022-04-11 15:54:53.521303', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='2', Assignmentid='2',Time='2022-04-11 15:54:53.531926', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='3', Assignmentid='3',Time='2022-04-11 15:54:53.540394', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='4', Assignmentid='4',Time='2022-04-11 15:54:53.548191', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='5', Assignmentid='5',Time='2022-04-11 15:54:53.555886', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='6', Assignmentid='6',Time='2022-04-11 15:54:53.567466', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='7', Assignmentid='7',Time='2022-0

In [15]:
# Answers generator
def Ans_generator(size=100,s=123):
    np.random.seed(s)
    r=np.random.randint(1,100,size).astype(str)
    Answer=pd.DataFrame(columns=['id','Text'])
    id=[]
    text=[]
    for i in range(len(r)):
        id.append(r[i])
        text.append('Example Answer text '+r[i]+': My solution is ......')
    Answer['id']=id
    Answer['Text']=text
    return(Answer)

Ans=Ans_generator()

In [16]:
# add  part of students' answers into DB
for i in range(90):
    db.addAnswer(Q_id=[1,2,3,4,5,6,7],universityid=uid[i],text=Ans.loc[i][1])

In [17]:
# generate Score of Questions
def Score_generator(mean=6,sd=3,size=100,n=7,s=123):
    '''default: call this function return 1 random num from Normal(6,9)'''
    '''defult random seed = 123'''
    '''every student has 7 question answers'''
    np.random.seed(s)
    r=np.random.normal(mean,sd,(100,7))
    for i in range(len(r)):
        for j in range(r.shape[1]):
            if r[i,j]>10:
                r[i,j]=round(10,1)
            elif r[i,j]<0:
                r[i,j]=round(0,1)
            else:
                r[i,j]=round(r[i,j],1)
    return r

ran_score=Score_generator()

In [18]:
for i in range(len(uid)):
    db.addScore(uid[i],ran_score[i])

C:\Users\Administrator\Desktop\pywork\DB\gradedb.py:115: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  a = Score(Submissionid=submissionid,Answerid=answeridlist[i],Value=value[i],Time=pd.datetime.now())


In [19]:
#view newsub
engine0 = create_engine("sqlite:///create.db",echo=False)
session = Session(engine0)
stmt = select(Score)

for user in session.scalars(stmt):
    print(user)

Score(Scoreid= '1', Answerid='1', Submissionid='1',Value='2.7',Time='2022-04-11 15:54:59.954477')
Score(Scoreid= '2', Answerid='2', Submissionid='1',Value='9.0',Time='2022-04-11 15:54:59.962944')
Score(Scoreid= '3', Answerid='3', Submissionid='1',Value='6.8',Time='2022-04-11 15:54:59.969423')
Score(Scoreid= '4', Answerid='4', Submissionid='1',Value='1.5',Time='2022-04-11 15:54:59.974343')
Score(Scoreid= '5', Answerid='5', Submissionid='1',Value='4.3',Time='2022-04-11 15:54:59.982065')
Score(Scoreid= '6', Answerid='6', Submissionid='1',Value='10.0',Time='2022-04-11 15:54:59.988639')
Score(Scoreid= '7', Answerid='7', Submissionid='1',Value='0.0',Time='2022-04-11 15:54:59.997765')
Score(Scoreid= '8', Answerid='8', Submissionid='2',Value='4.7',Time='2022-04-11 15:55:00.013684')
Score(Scoreid= '9', Answerid='9', Submissionid='2',Value='9.8',Time='2022-04-11 15:55:00.019913')
Score(Scoreid= '10', Answerid='10', Submissionid='2',Value='3.4',Time='2022-04-11 15:55:00.025374')
Score(Scoreid= '1

<ipython-input-19-2797ad2193b7>:6: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  for user in session.scalars(stmt):
